<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/gridVeg_groundCover_intercepts_WRANGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

In [ ]:
# Package and library installation
packages_needed = c("tidyverse", "knitr") # comma delimited vector of package names
packages_installed = packages_needed %in% rownames(installed.packages())

if (any(! packages_installed))
  install.packages(packages_needed[! packages_installed])
for (i in 1:length(packages_needed)) {
  library(packages_needed[i], character.only = T)
}

* Remember that the file containing authorization keys for Big Query must be loaded into the virutual envrionment manually.

In [ ]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Source

## Database Connection

In [ ]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [ ]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [ ]:
billing <- bq_test_project()

### vegetation_point_intercept_gridVeg

In [ ]:
con_point_intercept <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_point_intercept_gridVeg",
  billing = billing
)

In [ ]:
dbListTables(con_point_intercept)

[1] "gridVeg_additional_species"         "gridVeg_ground_cover_metadata"     
[3] "gridVeg_image_metadata"             "gridVeg_point_intercept_ground"    
[5] "gridVeg_point_intercept_vegetation" "gridVeg_shrub_tree"                
[7] "gridVeg_survey_metadata"

In [ ]:
ground_point_intercept_sql <- 
"
  SELECT
    survey_ID,
    grid_point,
    intercept_ground_code,
    COUNT(intercept_ground_code) / 2 AS intercepts_pct
  FROM 
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_point_intercept_ground`
  GROUP BY
    survey_ID, grid_point, intercept_ground_code
"
bq_ground_point_intercept <- bq_project_query(billing, ground_point_intercept_sql)
tb_ground_point_intercept <- bq_table_download(bq_ground_point_intercept)
# Drop NA rows. These only occur in intercept_ground_code, are always associated with 
# zero values in intercepts_pct, and will be expanded to true values later using complete()
df_ground_point_intercept <- as.data.frame(tb_ground_point_intercept) %>% drop_na() %>% glimpse()

Rows: 8,599
Columns: 4
$ survey_ID             <chr> "69", "234", "236", "237", "238", "239", "249",…
$ grid_point            <int> 329, 48, 295, 209, 61, 68, 67, 40, 41, 208, 145…
$ intercept_ground_code <chr> "G", "G", "G", "G", "G", "G", "G", "G", "G", "G…
$ intercepts_pct        <dbl> 2.0, 16.5, 17.5, 6.0, 16.0, 1.5, 4.0, 3.5, 1.0,…


### gridVeg_survey_metadata

In [ ]:
survey_metadata_sql <-
"
  SELECT
    survey_ID,
    year,
    survey_sequence
  FROM
    `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata`
"
bq_survey_metadata <- bq_project_query(billing, survey_metadata_sql)
tb_survey_metadata <- bq_table_download(bq_survey_metadata)
df_survey_metadata <- as.data.frame(tb_survey_metadata) %>% glimpse()

Rows: 1,472
Columns: 3
$ survey_ID       <chr> "F31C56A8-912D-410C-A17D-4C2DD75F71A4", "A19E87E6-A89…
$ year            <int> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016,…
$ survey_sequence <chr> "2016", "2016", "2016", "2016", "2016", "2016", "2016…


### ground_cover_metadata

In [ ]:
sql_ground_meta <- 
"
SELECT
  intercept_ground_code,
  ground_group
FROM
  `mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_ground_cover_metadata`
"
bq_ground_meta <- bq_project_query(billing, sql_ground_meta)
tb_ground_meta <- bq_table_download(bq_ground_meta)
df_ground_meta <- as.data.frame(tb_ground_meta) %>% arrange(intercept_ground_code) %>% glimpse()

Rows: 18
Columns: 2
$ intercept_ground_code <chr> "BG", "BV", "G", "L", "LIC", "M", "M/L", "OTHER…
$ ground_group          <chr> "inorganic", "vas_plant", "inorganic", "litter"…


# Wrangle

In [ ]:
# show unique intercept ground codes
df_ground_point_intercept %>%
  distinct(intercept_ground_code) %>% 
  arrange(intercept_ground_code)

intercept_ground_code
<chr>
BG
BV
G
L
LIC
M
M/L
OTHER
R


## Complete intercept_ground_code
- Complete the data frame for `intercept_ground_code` and fill null cells with 0
- As of 2020-08-24: with 18 ground cover codes and 1244 survey IDs, we should end up with 22,392 rows after completeing the data frame
- We should also end up with 1244 distinct combinations of survey_ID and grid_point

In [ ]:
df_ground_point_intercept_fill <- 
  df_ground_point_intercept %>%
  complete(intercept_ground_code, nesting(survey_ID, grid_point), fill= list(intercepts_pct = 0)) %>%
  glimpse()

df_ground_point_intercept_fill %>% 
  distinct(survey_ID, grid_point) %>% 
  count()

Rows: 22,392
Columns: 4
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG",…
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "013380…
$ grid_point            <int> 285, 505, 401, 23, 323, 511, 402, 196, 226, 564…
$ intercepts_pct        <dbl> 15.5, 14.5, 16.5, 16.5, 0.0, 0.0, 19.5, 37.0, 3…


n
<int>
1244


## Join datasets

In [ ]:
df_gridVeg_groundCover_intercepts <- 
  df_ground_point_intercept_fill %>%
  left_join(df_survey_metadata, by = "survey_ID") %>% 
  left_join(df_ground_meta, by = "intercept_ground_code") %>% 
  select(survey_ID, year, survey_sequence, grid_point, intercept_ground_code, ground_group, intercepts_pct) %>% 
  glimpse()

Rows: 22,392
Columns: 7
$ survey_ID             <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "013380…
$ year                  <int> 2016, 2016, 2017, 2016, 2015, 2016, 2015, 2016,…
$ survey_sequence       <chr> "2016", "2016", "2017", "2016", "2015", "2016",…
$ grid_point            <int> 285, 505, 401, 23, 323, 511, 402, 196, 226, 564…
$ intercept_ground_code <chr> "BG", "BG", "BG", "BG", "BG", "BG", "BG", "BG",…
$ ground_group          <chr> "inorganic", "inorganic", "inorganic", "inorgan…
$ intercepts_pct        <dbl> 15.5, 14.5, 16.5, 16.5, 0.0, 0.0, 19.5, 37.0, 3…


## QC
* No rows contain missing data, and this makes sense since `complete()` was used to fill response variables.
* Numeric variables appear correct in the summary

In [ ]:
df_gridVeg_groundCover_intercepts[which(!complete.cases(df_gridVeg_groundCover_intercepts)), ]

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>


In [ ]:
summary(df_gridVeg_groundCover_intercepts)

  survey_ID              year      survey_sequence      grid_point   
 Length:22392       Min.   :2011   Length:22392       Min.   :  1.0  
 Class :character   1st Qu.:2011   Class :character   1st Qu.:136.8  
 Mode  :character   Median :2015   Mode  :character   Median :290.0  
                    Mean   :2014                      Mean   :286.3  
                    3rd Qu.:2016                      3rd Qu.:431.0  
                    Max.   :2017                      Max.   :583.0  
 intercept_ground_code ground_group       intercepts_pct  
 Length:22392          Length:22392       Min.   : 0.000  
 Class :character      Class :character   1st Qu.: 0.000  
 Mode  :character      Mode  :character   Median : 0.000  
                                          Mean   : 5.545  
                                          3rd Qu.: 2.000  
                                          Max.   :99.500  

In [ ]:
df_gridVeg_groundCover_intercepts %>% filter(survey_ID == 234)

survey_ID,year,survey_sequence,grid_point,intercept_ground_code,ground_group,intercepts_pct
<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>
234,2011,2011-12,48,BG,inorganic,6.5
234,2011,2011-12,48,BV,vas_plant,19.5
234,2011,2011-12,48,G,inorganic,16.5
234,2011,2011-12,48,L,litter,41.5
234,2011,2011-12,48,LIC,lichen,0.0
234,2011,2011-12,48,M,nonvas_plant,0.0
234,2011,2011-12,48,M/L,nonvas_plant,12.0
234,2011,2011-12,48,OTHER,inorganic,0.0
234,2011,2011-12,48,R,inorganic,3.0


# Output

In [ ]:
write_csv(df_gridVeg_groundCover_intercepts, path = "gridVeg_groundCover_intercepts_WRANGLE.csv")